In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Window
import pandas as pd
import pandas_redshift as pr

In [2]:
# pip install pandas_redshift
spark = SparkSession.builder.getOrCreate()

In [3]:
pr.connect_to_redshift(
    host= 'bootcamp-cluster-1.c8mq2kyighu9.us-east-2.redshift.amazonaws.com',
    user = 'bootcamp_user',
    port = '5439',
    password = 'Bootcampuser20',
    dbname='dev'
)

In [4]:
sql_statement = 'select * from driverdim_hamzah_temp'
sql_statement1 = 'select * from driverfeed_hamzah'
sql_statement2 = 'select * from shifttimedim_hamzah'
sql_statement3 = 'select * from coveragedim_hamzah'
sql_statement4 = 'select * from companydim_hamzah'
sql_statement5 = 'select * from skyline_test_Hamzah'


In [5]:
df = pr.redshift_to_pandas(sql_statement)
df1 = pr.redshift_to_pandas(sql_statement1)
df2 = pr.redshift_to_pandas(sql_statement2)
df3 = pr.redshift_to_pandas(sql_statement3)
df4 = pr.redshift_to_pandas(sql_statement4)
df5 = pr.redshift_to_pandas(sql_statement5)

In [6]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [7]:
#These are the five dataframes
pickupFact = spark.createDataFrame(df5)
driverdim = spark.createDataFrame(df)
driverfeed = spark.createDataFrame(df1)
shiftTime = spark.createDataFrame(df2)
coverage = spark.createDataFrame(df3)
company = spark.createDataFrame(df4)

In [8]:
maxx = driverdim.agg({"driverkey": "max"}).head(1)[0][0]

In [16]:
def deleteFromDrivedim(drivedim, driverfeed): 
    drivedimFired = driverdim.join(driverfeed, (driverdim.drivername == driverfeed.drivername) & (driverfeed.action == 'FIRED'), how='inner') 
    drivedimFired = drivedimFired.select(['driverkey', driverdim.drivername, 'shifttimekey', 'coveragekey','companykey'])
    
    return drivedimFired 
drivedimFired = deleteFromDrivedim(driverdim, driverfeed)
drivedimRemoved = driverdim.join(drivedimFired, (drivedimFired.drivername == driverdim.drivername),how='left_anti')

In [12]:
drivedimRemoved.show()

+---------+----------+------------+-----------+----------+
|driverkey|drivername|shifttimekey|coveragekey|compnaykey|
+---------+----------+------------+-----------+----------+
|       26|   F02Name|           1|          8|         2|
|       25|   F01Name|           1|          2|         2|
|       35|   F11Name|           2|          6|         2|
|       43|   F19Name|           3|          1|         2|
|       32|   F08Name|           1|          1|         2|
|       41|   F17Name|           3|          3|         2|
|       38|   F14Name|           2|          3|         2|
|       69|   L21Name|           3|          4|         3|
|       68|   L20Name|           3|          8|         3|
|       72|   L24Name|           3|          7|         3|
|       51|   L03Name|           1|          7|         3|
|       57|   L09Name|           2|          2|         3|
|       71|   L23Name|           3|          5|         3|
|       58|   L10Name|           2|          1|         

In [17]:
def changeAttr(drivedim, drivefeed, shiftime, cove, comp):
    drivedimChanged=  drivedim.join(driverfeed, (drivedim.drivername == drivefeed.drivername) & (drivefeed.action == 'CHANGE'), how='inner').join(shiftime, (drivefeed.beginshift==shiftime.starttime) & (drivefeed.endshift==shiftime.endtime), how='inner').join(cove, (drivefeed.coverage==cove.coverage), how='inner').join(comp, (drivefeed.source==comp.company), how='inner')
    drivedimChanged = drivedimChanged.select(['driverkey', drivedim.drivername, shiftime.shifttimekey, cove.coveragekey,comp.compnaykey])
    
    return drivedimChanged
drivedimChanged = changeAttr(drivedimRemoved, driverfeed, shiftTime, coverage, company)

In [18]:
drivedimChanged.show()

+---------+----------+------------+-----------+----------+
|driverkey|drivername|shifttimekey|coveragekey|compnaykey|
+---------+----------+------------+-----------+----------+
|       26|   F02Name|           1|          8|         2|
|       25|   F01Name|           1|          2|         2|
|       35|   F11Name|           2|          6|         2|
|       43|   F19Name|           3|          1|         2|
|       32|   F08Name|           1|          1|         2|
|       41|   F17Name|           3|          3|         2|
|       38|   F14Name|           2|          3|         2|
|       69|   L21Name|           3|          4|         3|
|       68|   L20Name|           3|          8|         3|
|       72|   L24Name|           3|          7|         3|
|       51|   L03Name|           1|          7|         3|
|       57|   L09Name|           2|          2|         3|
|       71|   L23Name|           3|          5|         3|
|       58|   L10Name|           2|          1|         

In [19]:
intersecting = driverdim.join(drivedimChanged, (drivedimChanged.drivername == driverdim.drivername), how='left_anti')
drivedimChanged = drivedimChanged.unionAll(intersecting)
drivedimRemoved = drivedimChanged.join(drivedimFired, (drivedimFired.drivername == drivedimChanged.drivername),how='left_anti')

In [20]:
def hired(drivedim, driverfeed, shiftime, cove, comp):
    drivedimChanged=  driverfeed.join(shiftime, (driverfeed.action == 'HIRED') & (driverfeed.beginshift==shiftime.starttime) & (driverfeed.endshift==shiftime.endtime), how='inner').join(cove, (driverfeed.coverage==cove.coverage), how='inner').join(comp, (driverfeed.source==comp.company), how='inner')
    drivedimChanged = drivedimChanged.withColumn('key', F.row_number().over(Window.orderBy(F.monotonically_increasing_id())))
    
    drivedimChanged = drivedimChanged.withColumn('driverkey', maxx+drivedimChanged.key)
    drivedimChanged = drivedimChanged.select(['driverkey', driverfeed.drivername,shiftime.shifttimekey, cove.coveragekey,comp.compnaykey])
    return drivedimChanged
fd = hired(driverdim, driverfeed, shiftTime, coverage, company)
driverAdded = drivedimRemoved.union(fd)
driverAdded = driverAdded.orderBy('driverkey', ascending=True)

In [21]:
def removeTrailingAndLeadingSpaces(df):
    for x in df.columns:
         df = df.withColumn(x, F.trim(x))
    return df

In [22]:
def standardizeTime(df):
    pickupFactTime = df.withColumn('time', df.datetime.substr(-5, 5))
    pickupFactTime = removeTrailingAndLeadingSpaces(pickupFactTime)
    pickupFactTime = pickupFactTime.withColumn('time', F.concat(F.lit('0'),pickupFactTime.time))
    pickupFactTime = pickupFactTime.withColumn('time', pickupFactTime.time.substr(-5, 5))
    return pickupFactTime
pickupFactTime = standardizeTime(pickupFact)

In [23]:
def checkDates(strCheck):
    
    if (strCheck > '00:00') & (strCheck <= '07:59'):
        return 1
    elif (strCheck >= '08:00') & (strCheck <= '15:59'):
        return 2
    elif (strCheck >= '16:00') & (strCheck <= '23:59'):
        return 3
    return -1

udf_time = F.udf(checkDates, T.IntegerType())
pickupFactTime = pickupFactTime.withColumn('SKey', udf_time(F.col('time')))

In [24]:
def checkCoverge(string):
    coverage = {'Bronx': 1, 'Brooklyn': 2, 'JFK': 3, 'La Guardia': 4, 'Manhattan': 5, 'New York City': 6, 'Newark': 7, 'Other':8}
    for key in coverage:
        if key == string:
            return coverage.get(key)
    return -1
udf_coverage = F.udf(checkCoverge, T.IntegerType())
pickupFactTime = pickupFactTime.withColumn('CoveKey', udf_coverage(F.col('city')))

In [25]:
def checkCompany(string):
    company = {'Dial7': 1, 'First Class': 2, 'Lyft': 3, 'Skyline':4 }
    for key in company:
        if key == string:
            return company.get(key)
        
udf_company = F.udf(checkCompany, T.IntegerType())
pickupFactTime = pickupFactTime.withColumn('CompKey', udf_company(F.col('source')))

In [26]:
pickupFactTime = pickupFactTime.withColumn("DriverKey", F.lit(1))
pickupFactTime = pickupFactTime.join(driverAdded, (driverAdded.shifttimekey==pickupFactTime.SKey)&(driverAdded.coveragekey==pickupFactTime.CoveKey)&(driverAdded.compnaykey==pickupFactTime.CompKey))
pickupFactTime = pickupFactTime.select([driverAdded.driverkey, 'datetime', 'street_address', 'city', 'state', 'source']).persist()

In [27]:
pickupFactTime.show()

+---------+--------------+-------------------+----------+--------+-------+
|driverkey|      datetime|     street_address|      city|   state| source|
+---------+--------------+-------------------+----------+--------+-------+
|       92|7/1/2014 17:45|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 17:30|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 16:45|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 19:10|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 20:55|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 16:25|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 21:05|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 23:00|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 17:20|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 20:40|102-05 DITMARS BLVD|La Guardia|NEW YORK|Skyline|
|       92|7/1/2014 19:05

In [ ]:
pickupFactTime.coalesce(1).write.csv('Fct_withKeys.csv')

In [ ]:
pr.close_up_shop()